In [1]:
import wfdb
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import os
import json
import tqdm
import numpy as np

In [2]:
pd.set_option("display.max_columns", None)

path_data_csv = r"C:\Users\magra\Documents\HSD\5_Semester\Advances_AI\Project\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"
ptbxl = pd.read_csv(os.path.join(path_data_csv, "ptbxl_database.csv"))
ptbxl.head()

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,heart_axis,infarction_stadium1,infarction_stadium2,validated_by,second_opinion,initial_autogenerated_report,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


In [3]:
ptbxl_ann = ptbxl[["ecg_id", "patient_id", "scp_codes", "filename_lr"]]
ptbxl_ann.head()

,ecg_id,patient_id,scp_codes,filename_lr
0,1,15709.0,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",records100/00000/00001_lr
1,2,13243.0,"{'NORM': 80.0, 'SBRAD': 0.0}",records100/00000/00002_lr
2,3,20372.0,"{'NORM': 100.0, 'SR': 0.0}",records100/00000/00003_lr
3,4,17014.0,"{'NORM': 100.0, 'SR': 0.0}",records100/00000/00004_lr
4,5,17448.0,"{'NORM': 100.0, 'SR': 0.0}",records100/00000/00005_lr


In [4]:
intermediary_store=[]
import json
for index, row in ptbxl_ann.iterrows():
    row["scp_codes"] = row["scp_codes"].replace("'", '"')
    row["scp_codes"] = json.loads(row["scp_codes"])
    intermediary_store.append(row["scp_codes"])
intermediary_store

[{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0},
 {'NORM': 80.0, 'SBRAD': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 80.0, 'SARRH': 0.0},
 {'NORM': 80.0, 'SBRAD': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SARRH': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0},
 {'AFLT': 100.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'AFLT': 100.0, 'ABQRS': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NDT': 100.0, 'SR': 0.0},
 {'AFLT': 100.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0},
 {'NORM': 80.0, 'SR': 0.0},
 {'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'LVH': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'LPFB': 100.0, 'SR': 0.0},
 {'NORM': 100.0

In [5]:
ptbxl_ann["scp_codes"] = intermediary_store

C:\Users\magra\AppData\Local\Temp\ipykernel_10296\1873181576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptbxl_ann["scp_codes"] = intermediary_store


In [6]:
data_one_scp_code = []
for index, row in ptbxl_ann.iterrows():
    dictionary = row["scp_codes"]
    
    
    if isinstance(dictionary, dict):
        print(dictionary)
        # find key that corresponds to highest value
        max_key = max(dictionary, key = dictionary.get)
        print(max_key)
        # get value associated with max_key
        max_value = dictionary[max_key]

        if max_value >= 70:
            # first two columns stay the same
            ecg_id_value = row["ecg_id"]
            patient_id_value = row["patient_id"]
            filename_100 = row["filename_lr"]
            # add to list for storage
            data_one_scp_code.append([ecg_id_value, patient_id_value, max_key, max_value, filename_100])

data_one_scp_df = pd.DataFrame(data_one_scp_code, columns=["ecg_id", "patient_id", "max_key", "max_value", "filename_lr"])

{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
NORM
{'NORM': 80.0, 'SBRAD': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0}
IMI
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 80.0, 'SARRH': 0.0}
NORM
{'NORM': 80.0, 'SBRAD': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SARRH': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
AFLT
{'AFLT': 100.0}
AFLT
{'NORM': 100.0, 'SR': 0.0}
NORM
{'AFLT': 100.0, 'ABQRS': 0.0}
AFLT
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NDT': 100.0, 'SR': 0.0}
NDT
{'AFLT': 100.0}
AFLT
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NORM': 100.0, 'SR': 0.0}
NORM
{'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0}
NST_
{'NORM': 80.0, 'SR': 0.0}
NORM
{'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0}
NST_
{'NORM': 100.0, 'SR': 0.0}
NORM
{'LVH': 100.0, '

In [7]:
data_one_scp_df
print(data_one_scp_df)


       ecg_id  patient_id max_key  max_value                filename_lr
0           1     15709.0    NORM      100.0  records100/00000/00001_lr
1           2     13243.0    NORM       80.0  records100/00000/00002_lr
2           3     20372.0    NORM      100.0  records100/00000/00003_lr
3           4     17014.0    NORM      100.0  records100/00000/00004_lr
4           5     17448.0    NORM      100.0  records100/00000/00005_lr
...       ...         ...     ...        ...                        ...
19678   21832      7954.0    LAFB      100.0  records100/21000/21832_lr
19679   21833     17180.0     NDT      100.0  records100/21000/21833_lr
19680   21834     20703.0    NORM      100.0  records100/21000/21834_lr
19681   21836      8873.0    NORM      100.0  records100/21000/21836_lr
19682   21837     11744.0    NORM      100.0  records100/21000/21837_lr

[19683 rows x 5 columns]


In [8]:
prelim_featurevector = pd.DataFrame(data_one_scp_df).to_numpy()
print(prelim_featurevector)
print(type(prelim_featurevector))
print(prelim_featurevector.shape)

[[1 15709.0 'NORM' 100.0 'records100/00000/00001_lr']
 [2 13243.0 'NORM' 80.0 'records100/00000/00002_lr']
 [3 20372.0 'NORM' 100.0 'records100/00000/00003_lr']
 ...
 [21834 20703.0 'NORM' 100.0 'records100/21000/21834_lr']
 [21836 8873.0 'NORM' 100.0 'records100/21000/21836_lr']
 [21837 11744.0 'NORM' 100.0 'records100/21000/21837_lr']]
<class 'numpy.ndarray'>
(19683, 5)


In [9]:
prelim_featurevector[0][4]

'records100/00000/00001_lr'

In [10]:
path_wfdb = r"C:\Users\magra\Documents\HSD\5_Semester\Advances_AI\Project\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"
#path_wfdb = path_wfdb.replace('\\', "/")
path_wfdb

'C:\\Users\\magra\\Documents\\HSD\\5_Semester\\Advances_AI\\Project\\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3'

In [11]:
wfdb_data = []
for i in tqdm.tqdm(range(prelim_featurevector.shape[0])):
    #print(prelim_featurevector[i][4])
    path_wfdb_100 = os.path.join(path_wfdb,  prelim_featurevector[i][4])
    #path_wfdb_100
    record_100 = wfdb.rdrecord(path_wfdb_100)
    signal_physical_100 = record_100.p_signal
    
    wfdb_data.append(signal_physical_100)
    #print(wfdb_data)

  0%|          | 0/19683 [00:00<?, ?it/s]

100%|██████████| 19683/19683 [04:40<00:00, 70.05it/s] 


In [12]:
len(wfdb_data)

19683

In [13]:
wfdb_data_np = np.array(wfdb_data)

In [14]:
wfdb_data_np.shape

(19683, 1000, 12)

In [17]:
prelim_featurevector_df = pd.DataFrame(prelim_featurevector)
prelim_featurevector_df

,0,1,2,3,4
0,1,15709.0,NORM,100.0,records100/00000/00001_lr
1,2,13243.0,NORM,80.0,records100/00000/00002_lr
2,3,20372.0,NORM,100.0,records100/00000/00003_lr
3,4,17014.0,NORM,100.0,records100/00000/00004_lr
4,5,17448.0,NORM,100.0,records100/00000/00005_lr
...,...,...,...,...,...
19678,21832,7954.0,LAFB,100.0,records100/21000/21832_lr
19679,21833,17180.0,NDT,100.0,records100/21000/21833_lr
19680,21834,20703.0,NORM,100.0,records100/21000/21834_lr
19681,21836,8873.0,NORM,100.0,records100/21000/21836_lr


In [18]:
prelim_featurevector_wfdb_df["wfdb"] = wfdb_data_np.tolist()

In [19]:
prelim_featurevector_wfdb_df

,0,1,2,3,4,wfdb
0,1,15709.0,NORM,100.0,records100/00000/00001_lr,"[[-0.119, -0.055, 0.064, 0.086, -0.091, 0.004,..."
1,2,13243.0,NORM,80.0,records100/00000/00002_lr,"[[0.004, 0.138, 0.134, -0.072, -0.065, 0.136, ..."
2,3,20372.0,NORM,100.0,records100/00000/00003_lr,"[[-0.029, -0.079, -0.049, 0.054, 0.011, -0.064..."
3,4,17014.0,NORM,100.0,records100/00000/00004_lr,"[[-0.054, -0.138, -0.083, 0.096, 0.015, -0.11,..."
4,5,17448.0,NORM,100.0,records100/00000/00005_lr,"[[-0.034, -0.574, -0.54, 0.304, 0.253, -0.556,..."
...,...,...,...,...,...,...
19678,21832,7954.0,LAFB,100.0,records100/21000/21832_lr,"[[-0.076, -0.029, 0.046, 0.052, -0.061, 0.008,..."
19679,21833,17180.0,NDT,100.0,records100/21000/21833_lr,"[[-0.052, -0.034, 0.018, 0.043, -0.034, -0.008..."
19680,21834,20703.0,NORM,100.0,records100/21000/21834_lr,"[[-0.05, -0.013, 0.036, 0.031, -0.042, 0.011, ..."
19681,21836,8873.0,NORM,100.0,records100/21000/21836_lr,"[[-0.057, -0.057, 0.0, 0.057, -0.028, -0.028, ..."


In [23]:
prelim_featurevector_wfdb_np = prelim_featurevector_wfdb_df.to_numpy()

In [24]:
prelim_featurevector_wfdb_np.shape

(19683, 6)

In [25]:
prelim_featurevector_wfdb_np

array([[1, 15709.0, 'NORM', 100.0, 'records100/00000/00001_lr',
        list([[-0.119, -0.055, 0.064, 0.086, -0.091, 0.004, -0.069, -0.031, 0.0, -0.026, -0.039, -0.079], [-0.116, -0.051, 0.065, 0.083, -0.09, 0.006, -0.064, -0.036, -0.003, -0.031, -0.034, -0.074], [-0.12, -0.044, 0.076, 0.082, -0.098, 0.016, -0.058, -0.034, -0.01, -0.028, -0.029, -0.069], [-0.117, -0.038, 0.08, 0.077, -0.098, 0.021, -0.05, -0.03, -0.015, -0.023, -0.022, -0.064], [-0.103, -0.031, 0.072, 0.066, -0.087, 0.021, -0.045, -0.027, -0.02, -0.019, -0.018, -0.058], [-0.097, -0.025, 0.071, 0.061, -0.084, 0.023, -0.036, -0.025, -0.009, -0.014, -0.012, -0.052], [-0.119, -0.014, 0.106, 0.066, -0.112, 0.046, -0.029, -0.012, 0.005, -0.008, -0.007, -0.048], [-0.096, 0.008, 0.104, 0.044, -0.1, 0.056, -0.023, 0.003, 0.018, 0.002, -0.001, -0.041], [-0.048, 0.044, 0.092, 0.002, -0.07, 0.068, -0.015, 0.018, 0.021, 0.009, 0.005, -0.038], [-0.037, 0.045, 0.081, -0.004, -0.059, 0.063, -0.05, 0.009, 0.018, 0.022, 0.009, -0.033], 

In [76]:
#wfdb_data_np = wfdb_data_np.reshape(19683, -1)
prelim_featurevector_wfdb = np.concatenate((prelim_featurevector, wfdb_data_np[:,:,:].reshape((19683, -1))), axis=1)

In [74]:
prelim_featurevector_wfdb = np.hstack((prelim_featurevector, wfdb_data_np))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [71]:
prelim_featurevector_wfdb.shape

(19683, 12005)

In [ ]:
print(path_wfdb)
path_wfdb_100 = os.path.join(path_wfdb,  prelim_featurevector[0][4])
path_wfdb_100

In [ ]:
record = wfdb.rdrecord(path_wfdb_100)
wfdb.plot_wfdb(record=record, title='Trial, ECG, Ending .hea') 
display(record.__dict__)

In [ ]:
signal_physical = record.p_signal
print(signal_physical)
signal_physical.shape